# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

Matplotlib is building the font cache; this may take a moment.


In [3]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [5]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [8]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [9]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [10]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

## Матрица рейтингов

In [11]:
from scipy.sparse import coo_matrix, csr_matrix

In [12]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [13]:
R

<COOrdinate sparse matrix of dtype 'float64'
	with 855598 stored elements and shape (2113, 10109)>

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [14]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [15]:
user_1 = R[0]
user_1

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 55 stored elements and shape (1, 10109)>

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [16]:
user_2 = R[1]
user_2

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 468 stored elements and shape (1, 10109)>

Мы можем сравнивать элементы с 0

In [17]:
user_1_rated = (user_1 != 0)
user_1_rated

<Compressed Sparse Row sparse matrix of dtype 'bool'
	with 55 stored elements and shape (1, 10109)>

Можем их "индексировать"

In [18]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [19]:
user_1.nnz

55

Можем умножать 2 разреженных вектора поэлементно:

In [20]:
(user_1).multiply(user_2)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14 stored elements and shape (1, 10109)>

и скалярно

In [21]:
user_1.dot(user_2.T)[0, 0]

np.float64(216.75)

И превращать разреженную матрицу (вектор) в плотную

In [22]:
user_1_dense = user_1.toarray()
user_1_dense

array([[0., 0., 1., ..., 0., 0., 0.]])

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


### _Решение_

In [23]:
def cosine_similarity_pair_users(u, v):
    u_rated = (u != 0)
    v_rated = (v != 0)
    common_items = (u_rated).multiply(v_rated)
    if common_items.nnz > 2:
        scalar = u.dot(v.T)[0, 0]
        norm = np.linalg.norm(u[common_items]) * np.linalg.norm(v[common_items])
        return scalar / norm
    else:
        return 0.0

In [24]:
answer1 = round(cosine_similarity_pair_users(R[146], R[239]),3)
print(answer1)

0.923


Введите значение answer1 на странице https://stepik.org/lesson/198104/step/6?unit=172340

## Функция нахождения пользователей, схожих с данным. 

Реализуйте функцию <font color = "blue">similar_users</font>(u, R, n_neigbours) которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей 
и возвращает отсортированный массив пользователей (сортировка по неубыванию), максимально похожих на данного. Для сортировки используйте np.argsort без параметров. (https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html) (Сам пользователь будет в этом списке на первом месте). Эту функцию вы сможете использовать далее. 

In [31]:
def similar_users(u, R, n_neigbours):
    n_users = R.shape[0]
    similarities = np.zeros(n_users)

    for i in range(n_users):
        similarities[i] = cosine_similarity_pair_users(R[u], R[i])

    sorted_users = np.argsort(-similarities)

    return sorted_users[:n_neigbours]

In [32]:
answer2 = np.array2string(similar_users(42, R, 10)).replace(' ','').replace('[','').replace(']','')

In [33]:
answer2

'42281633724815262065016921506'

Введите значение answer2 без кавычек  на странице https://stepik.org/lesson/198104/step/7?unit=172340

## Функция прогнозирования рейтинга

Реализуйте функцию <font color = "blue">rate_items_user</font>(u, R, n_neigbours), которая принимает на входе:
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей <font color = "red">(Теперь обратите внимание, несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)</font>

и возвращает вектор с предсказанными рейтингами по всем фильмам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [34]:
def rate_items_user(u, R, n_neigbours=30):
    predictions = csr_matrix((1, R.shape[1]))
    cumsim = 0.0
    num_predictions = csr_matrix((1, R.shape[1]))
    s = np.array([cosine_similarity_pair_users(R[u], R[v]) for v in range(R.shape[0])])
    similar_users = np.argsort(s)[::-1]
    print(np.sort(s)[::-1])
    for v in similar_users[:n_neigbours]:
        if v == u:
            continue
        user_sim = s[v]
        predictions += user_sim * R[v]
        cumsim += np.abs(user_sim)
    predictions /= cumsim
    return predictions

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).

* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

Т.е. предсказанные рейтинги можно получить так:
R_hat = <font color = "blue">rate_items_user</font>(20, R, n_neigbours=30). При сортировке фильмов по рейтингу используйте функцию <font color = "blue">argsort </font> без параметров.

### _Решение_

In [35]:
R_hat = rate_items_user(20, R, n_neigbours=30)

[1. 1. 1. ... 0. 0. 0.]


In [38]:
rated_items = (R[20] == 0)
unseen_ratings = R_hat.multiply(rated_items)
unseen_ratings = unseen_ratings.toarray()[0]

In [39]:
idx = unseen_ratings.argsort()[::-1]

In [40]:
unseen_ratings[idx]

array([2.5174164 , 2.24147475, 2.15522858, ..., 0.        , 0.        ,
       0.        ])

In [45]:
top5 = idx[:5]
top5

array([2614,  306,  343, 5573, 6720])

## Генерация ответа

In [43]:
answer3 = ', '.join(str(i) for i in top5)

In [44]:
answer3

'2614, 306, 343, 5573, 6720'

Полученную строку введите без кавычек на странице https://stepik.org/lesson/198104/step/8?unit=172340